### ANN Model trained on common features (intersection of random forest and decision tree)

In [1]:
#Import all the necessary libraries
import pandas as pd
import pandas as pd
import os 
import numpy as nm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential #importing Sequential Model.It uses tensorflow backend and you need to have tensorflow setup. 
from keras.layers import Dense #Importing default dense layer
from sklearn.utils import shuffle
from numpy.random import seed
seed(1)

path_dir='C:/Users/subhi/AMNS_Second_phase_ANN_model/Datasets/'
filename='Common_Data.xlsx'
#Access all the necessary variables and datasets
%store -r top_twenty_features_data 
%store -r top_features_data 
%store -r random_forest_data

In [2]:
#Finding common features in both datasets 
common_cols = top_twenty_features_data.columns.intersection(top_features_data.columns)
common_cols

Index(['NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
       'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN',
       'STEELLEVELINMOLD-SETMEAN', 'TUNDISHWEIGHTMEAN',
       'SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN', 'PER_NIMEAN',
       'SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN',
       'MOLDTAPERPERCENTMEAN', 'OUTPUT'],
      dtype='object')

In [3]:
#Accessing the datapoints
common_data=random_forest_data.loc[:,common_cols]
common_data

,NARROWFACESECONDARYCOOLINGWATERPRESSUREACTUALMEAN,MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN,STEELLEVELINMOLD-SETMEAN,TUNDISHWEIGHTMEAN,SEG0BSECONDARYCOOLINGWATERPRESSUREACTUALMEAN,PER_NIMEAN,SEG7+8TOPSECONDARYCOOLINGWATERPRESSUREACTUALMEAN,MOLDTAPERPERCENTMEAN,OUTPUT
0,4.925862,4.000000,80.103621,23.854885,1.177759,0.0150,2.462989,1.127,1.0
1,4.915455,4.000000,80.104899,23.548081,1.170657,0.0087,2.425101,1.127,1.0
2,5.482414,4.000000,80.106667,24.320115,1.239828,0.0146,2.693966,1.127,0.0
3,5.471203,4.000000,80.108861,24.275696,1.240000,0.0077,2.840443,1.127,0.0
4,5.081299,4.000000,80.110282,24.563785,1.190000,0.0067,2.613503,1.127,1.0
...,...,...,...,...,...,...,...,...,...
586,2.140476,7.994048,80.108333,23.541369,0.789524,0.0060,2.230060,0.884,0.0
587,2.098624,8.052910,80.107513,27.779471,0.334974,0.0178,1.479683,0.884,1.0
588,2.097216,7.407216,80.109175,22.814948,0.544330,0.0307,1.989381,0.884,1.0
589,2.091311,7.121359,80.106019,24.742087,0.722087,0.0234,0.650049,0.884,1.0


In [4]:
common_data.to_excel(filename)

#### Splitting the data into training and testing data

In [5]:
common_data = shuffle(common_data, random_state=42)
#Defining train and test sizes
total_length= common_data.shape[0]
train_size=int(common_data.shape[0]*0.8)
test_size=common_data.shape[0]-train_size

# First converting dataframe series into numpy array and setting the datatype as float32 for all data
# In second step converting all 1D array into 2D array
train_X=common_data.iloc[:,:-1][0:train_size].to_numpy().astype(nm.float32) #2D array

train_Y=common_data['OUTPUT'][0:train_size].to_numpy().astype(nm.float32) #1D array
train_Y=nm.reshape(train_Y,(train_Y.shape[0],1)) # 2D array

test_X=common_data.iloc[:,:-1][train_size:].to_numpy().astype(nm.float32) #2D array

test_Y=common_data['OUTPUT'][train_size:].to_numpy().astype(nm.float32) #1D array
test_Y=nm.reshape(test_Y,(test_Y.shape[0],1)) # 2D array

print('Dimension of training data',train_X.shape)
print('Dimension of testing data',test_X.shape)

#print(train_X)
print(type(train_X))
print(train_Y.shape)
test_Y.shape #2-D array
#train_Y

Dimension of training data (472, 8)
Dimension of testing data (119, 8)
<class 'numpy.ndarray'>
(472, 1)


(119, 1)

#### Scaling or normalising the data

In [6]:
scaler=MinMaxScaler(feature_range=(0,1)) # scaler object

# First fit the scaler with your training data.
# It will find those parameters using which it is going to scale.
# Then transform any data you want.

train_X=scaler.fit_transform(train_X) 

# Since it has scaled  the data as per the parameters it learnt during training data.
# So it will require 2D array/similiar dimension as of training data.
# So testing data should be a 2D array.
# Testing data only need to be transformed on the basis of the parameters learnt using fit function on training data.

test_X=scaler.transform(test_X) #Scaling the training data and using the same parameters for testing data
#test
#train_X[:]

#### Setting ANN model with regularization

In [7]:
from keras.regularizers import l2
#Creating a Model
print('Total No of features',len(train_X[0,:])) #No of features
model=Sequential() 

model.add(Dense(50,input_dim=len(train_X[0,:]),activation='relu',kernel_regularizer=l2(0.01))) # one hidden layer
model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer
        
model.summary()

Total No of features 8
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                450       
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 501
Trainable params: 501
Non-trainable params: 0
_________________________________________________________________


#### Compile the model

In [8]:
#model.compile(optimizer='_' , loss='_' , metrics=['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#### Fit the model

In [9]:
#Fitting the training data
#model.fit(training data input, training data output, batch_size=_,validation_data=(x_test,y_test))
model.fit(train_X,train_Y,epochs=150,batch_size=5,validation_data=(test_X,test_Y))

# Epochs is number of times i want to do forward and backward propogation.
# batch size is used for update the weights in mini batches. The default back size is 32.
# Validation data : accuracy looking like on validation data after every epoch, you can give test data here too. 
#The loss will be reduced due to backpropgation through time.

#Add regularization or increase number of epochs for better accuracy.

Epoch 1/150
95/95 [==============================] - 1s 3ms/step - loss: 0.8187 - accuracy: 0.5042 - val_loss: 0.7989 - val_accuracy: 0.4286
Epoch 2/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7734 - accuracy: 0.6081 - val_loss: 0.7673 - val_accuracy: 0.4622
Epoch 3/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7495 - accuracy: 0.5530 - val_loss: 0.7554 - val_accuracy: 0.4202
Epoch 4/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7340 - accuracy: 0.5487 - val_loss: 0.7342 - val_accuracy: 0.6050
Epoch 5/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7243 - accuracy: 0.6568 - val_loss: 0.7308 - val_accuracy: 0.4958
Epoch 6/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7174 - accuracy: 0.6589 - val_loss: 0.7238 - val_accuracy: 0.5546
Epoch 7/150
95/95 [==============================] - 0s 1ms/step - loss: 0.7130 - accuracy: 0.6377 - val_loss: 0.7157 - val_accuracy: 0.6218
Epoch 8/150
9

Epoch 59/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6900 - accuracy: 0.6377 - val_loss: 0.7000 - val_accuracy: 0.5546
Epoch 60/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6905 - accuracy: 0.6314 - val_loss: 0.7021 - val_accuracy: 0.5714
Epoch 61/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6901 - accuracy: 0.6059 - val_loss: 0.7069 - val_accuracy: 0.5714
Epoch 62/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6900 - accuracy: 0.6229 - val_loss: 0.6977 - val_accuracy: 0.5966
Epoch 63/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6900 - accuracy: 0.6335 - val_loss: 0.7034 - val_accuracy: 0.5630
Epoch 64/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6901 - accuracy: 0.6038 - val_loss: 0.6995 - val_accuracy: 0.5714
Epoch 65/150
95/95 [==============================] - 0s 2ms/step - loss: 0.6908 - accuracy: 0.6441 - val_loss: 0.6997 - val_accuracy: 0.5546
Epoch 

95/95 [==============================] - 0s 1ms/step - loss: 0.6895 - accuracy: 0.6335 - val_loss: 0.7020 - val_accuracy: 0.5630
Epoch 117/150
95/95 [==============================] - 0s 1ms/step - loss: 0.6892 - accuracy: 0.6504 - val_loss: 0.7014 - val_accuracy: 0.5714
Epoch 118/150
95/95 [==============================] - 0s 1ms/step - loss: 0.6897 - accuracy: 0.6483 - val_loss: 0.7023 - val_accuracy: 0.5546
Epoch 119/150
95/95 [==============================] - 0s 1ms/step - loss: 0.6893 - accuracy: 0.6335 - val_loss: 0.7030 - val_accuracy: 0.5462
Epoch 120/150
95/95 [==============================] - 0s 1ms/step - loss: 0.6889 - accuracy: 0.6504 - val_loss: 0.7035 - val_accuracy: 0.5462
Epoch 121/150
95/95 [==============================] - 0s 1ms/step - loss: 0.6892 - accuracy: 0.6504 - val_loss: 0.7038 - val_accuracy: 0.5546
Epoch 122/150
95/95 [==============================] - 0s 1ms/step - loss: 0.6902 - accuracy: 0.6377 - val_loss: 0.7041 - val_accuracy: 0.5546
Epoch 123/150